<a href="https://colab.research.google.com/github/BangAjus/Data-Analytics/blob/Data-Analysis/Prediksi_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing classic library for DS
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

# Importing models for analysis so we can pick the best model for our predictions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time

In [ ]:
from pandas import *

df = read_csv('diabet.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# how to check the duplicates
df[df.duplicated()]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [ ]:
df.isna().sum()

# if there are duplicates
# df.drop_duplicates(inplace = True)

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
# finding some columns with most correlation value
df.corr(method='pearson')['Outcome'].sort_values(ascending=False).iloc[1:7]

Glucose                     0.466581
BMI                         0.292695
Age                         0.238356
Pregnancies                 0.221898
DiabetesPedigreeFunction    0.173844
Insulin                     0.130548
Name: Outcome, dtype: float64

In [ ]:
X, y = df.drop(columns=['Outcome']), df['Outcome']
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.20, random_state=0)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
lr_model = LogisticRegression(max_iter = 10000)
ridge_model = RidgeClassifier()
ridgecv_model = RidgeClassifierCV()
gpc = GaussianProcessClassifier()
tr = tree.DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=3)
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
rf = RandomForestClassifier(max_depth=2)

In [ ]:
def model_metrics(model, X_test, y_test, decimals = 5):
    import numpy as np
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = np.round(accuracy_score(y_test, y_pred),decimals)
    pre = np.round(precision_score(y_test, y_pred),decimals)
    rec = np.round(recall_score(y_test, y_pred),decimals)
    f1 = np.round(f1_score(y_test, y_pred),decimals)
    auc = np.round(roc_auc_score(y_test, y_pred),decimals)
    return {'accuracy': acc, 'precision': pre, 'recall': rec, 'f1_score': f1, 'auc': auc}

In [ ]:
results = pd.DataFrame(
    [
        model_metrics(lr_model, X_test, y_test),
        model_metrics(ridge_model, X_test, y_test),
        model_metrics(ridgecv_model, X_test, y_test),
        model_metrics(gpc, X_test, y_test),
        model_metrics(tr, X_test, y_test),
        model_metrics(knn, X_test, y_test),
        model_metrics(svc, X_test, y_test),
        model_metrics(rf, X_test, y_test)
    ], 
    index = ['LogisticRegression', 'RidgeClassifier', 'RidgeClassifierCV', 'GaussianProcessClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier', 'SupportVectorClassification', 'RandomForestClassifier']) \
.reset_index() \
.rename(columns={'index':'model'})

In [ ]:
results.sort_values(['accuracy', 'f1_score', 'auc'],
              ascending = [False, False, False])

,model,accuracy,precision,recall,f1_score,auc
1,RidgeClassifier,0.83117,0.78378,0.61702,0.69048,0.77113
0,LogisticRegression,0.82468,0.76316,0.61702,0.68235,0.76645
2,RidgeClassifierCV,0.82468,0.77778,0.59574,0.67470,0.76049
4,DecisionTreeClassifier,0.79870,0.63793,0.78723,0.70476,0.79549
3,GaussianProcessClassifier,0.79221,0.68293,0.59574,0.63636,0.73712
6,SupportVectorClassification,0.79221,0.70270,0.55319,0.61905,0.72519
7,RandomForestClassifier,0.76623,0.70370,0.40426,0.51351,0.66474
5,KNeighborsClassifier,0.74026,0.57778,0.55319,0.56522,0.68781
